In [1]:
# !pip install langchain-chroma

In [3]:
# !pip install sentence-transformers

In [5]:
# !pip install -qU langchain-mistralai

In [2]:
# ASTRA_DB_APPLICATION_TOKEN
# ASTRA_DB_API_ENDPOINT
# HF_TOKEN
# MISTRALAI_API_KEY
# GROQ_API_KEY

In [2]:
!pip install "langchain-astradb>=0.1.0"

In [4]:
import os
from langchain import hub
import bs4
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import Tool
from langchain_groq import ChatGroq
from langchain_mistralai import MistralAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_astradb import AstraDBVectorStore
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

In [14]:
embeddings = MistralAIEmbeddings(api_key=api_key)

/opt/anaconda3/lib/python3.11/site-packages/langchain_mistralai/embeddings.py:180: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [16]:
os.getcwd()

'/Users/pranayyb/Documents/AIML/GenAI/Agents'

In [18]:
loader = DirectoryLoader("/Users/pranayyb/Documents/AIML/GenAI/Agents", glob = "./*.txt", loader_cls= TextLoader)
document=loader.load()

In [20]:
document

[Document(metadata={'source': '/Users/pranayyb/Documents/AIML/GenAI/Agents/Dating.txt'}, page_content='\ufeff"Is my girlfriend or boyfriend cheating on me?"\nThis is the absolute, number one relationship question of all time. If you go to Google and type in “Is my wife,” or “Is my husband,” Google will finish your sentence with “Cheating on me”. Sometimes, you don’t even get past the “my”!\nUnfortunately, science can’t provide a definitive answer to the question of cheating. It can, however, give us the tools to spot deception.\nDr. Paul Ekman, University of California Medical School emeritus psychology professor, has published something called the Facial Action Coding System—FACS—which successfully detects deception close to 90 percent of the time. Dr. Ekman says that the biggest indicators of lying come from the face, not the body—liars don’t actually fidget more than truthful people, despite what you may have heard.\nSome of Ekman’s biggest signs of deception are blinking, dilated p

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
text = text_splitter.split_documents(document)

In [24]:
text[170]

Document(metadata={'source': '/Users/pranayyb/Documents/AIML/GenAI/Agents/Dating.txt'}, page_content='Improving Communication Skills\nPatterns:\n                                                                                                      1. How can I improve my communication skills in dating to foster better connections?\nWhat specific techniques should I practice for effective dialogue?\n                                                                                                      2. What should I do if my partner and I argue frequently, and how can we resolve our conflicts?\nWhat steps can we take to communicate better during disagreements?\n                                                                                                      3. How can I express my feelings without causing conflict or misunderstanding?\nWhat are the best practices for sharing my emotions with my partner?\nResponses:')

In [115]:
# chunk_embeddings = [embeddings.embed_query(chunk.page_content) for chunk in text]

In [69]:
!pip install langchain_huggingface

In [28]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
chunk_embeddings=[embeddings.embed_query(chunk.page_content) for chunk in text]

In [30]:
len(chunk_embeddings)

176

In [32]:
documents=[]

In [34]:
for i, chunk in enumerate(chunk_embeddings):
    piece = {
        "text": text[i].page_content,
        "$vector": chunk
    }
    documents.append(piece)


In [36]:
documents

[{'text': '\ufeff"Is my girlfriend or boyfriend cheating on me?"\nThis is the absolute, number one relationship question of all time. If you go to Google and type in “Is my wife,” or “Is my husband,” Google will finish your sentence with “Cheating on me”. Sometimes, you don’t even get past the “my”!\nUnfortunately, science can’t provide a definitive answer to the question of cheating. It can, however, give us the tools to spot deception.\nDr. Paul Ekman, University of California Medical School emeritus psychology professor, has published something called the Facial Action Coding System—FACS—which successfully detects deception close to 90 percent of the time. Dr. Ekman says that the biggest indicators of lying come from the face, not the body—liars don’t actually fidget more than truthful people, despite what you may have heard.',
  '$vector': [-0.0693465992808342,
   -0.010173220187425613,
   0.05299663916230202,
   0.006052966229617596,
   -0.004397386219352484,
   -0.028828708454966

In [38]:
collection = AstraDBVectorStore(
    embedding=embeddings,
    collection_name="dateher2",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

In [42]:
collection

In [44]:
from langchain.schema import Document
import uuid
documents = [
    Document(id=str(uuid.uuid4()), page_content=doc["text"])
    for doc in documents
]

In [46]:
collection.add_documents(documents=documents)

['21fd3a13-cfd7-4919-93f9-65f39b058ab8',
 '27fbb32b-9b70-4db8-b964-7f3695f11e9e',
 'bff116cf-cd38-4702-941b-1da112e78125',
 '4248a6a5-46c9-47aa-92aa-4342ad5b5351',
 'a9b9f193-65f7-475e-9e61-df51223a3b8c',
 'be500ca9-ba85-40b4-be9c-fd885a60b21e',
 'ad0e7518-4507-4007-bda2-c10dd83032c7',
 '0cb3d865-eefc-48f0-88ed-34cfb7e99948',
 '67c62037-2715-4b36-81a0-e4c9a9a3c847',
 '4a2275bf-401e-4e63-8253-409df2972a74',
 'a6ca7df3-3b94-43f3-a46d-0dfc6bbcedb8',
 '53d617d0-b016-43ef-8881-ed05df6118a0',
 '60bf18f9-cb0c-4990-a133-450c1db3b4f9',
 '68e2e661-d4dd-43b8-b23c-2ae287f223b6',
 'c4f705b3-e93a-4303-8de7-03113aa8bcdf',
 '0dc3343a-db7a-4943-a5c7-15a7b78e2284',
 'b0363cd4-2152-47cc-afba-027b32675871',
 '691af3d6-d110-4de0-ab39-5a097d5c7944',
 '5312ce73-00ec-4239-a573-b0f5e67054e9',
 '0b53914e-311f-491c-9e17-269e68bcce46',
 'e1dafc6a-df1b-4c5c-9b35-fa216f520e1f',
 '5b90bdbb-378e-4a24-b5a4-4192ccd963c9',
 '14e40a2e-acba-4e35-a4ae-cc8d15c91c63',
 '03c7b351-c8d4-4578-b8f6-019611d50d25',
 '2dfc4304-a244-

In [48]:
documents

[Document(id='21fd3a13-cfd7-4919-93f9-65f39b058ab8', metadata={}, page_content='\ufeff"Is my girlfriend or boyfriend cheating on me?"\nThis is the absolute, number one relationship question of all time. If you go to Google and type in “Is my wife,” or “Is my husband,” Google will finish your sentence with “Cheating on me”. Sometimes, you don’t even get past the “my”!\nUnfortunately, science can’t provide a definitive answer to the question of cheating. It can, however, give us the tools to spot deception.\nDr. Paul Ekman, University of California Medical School emeritus psychology professor, has published something called the Facial Action Coding System—FACS—which successfully detects deception close to 90 percent of the time. Dr. Ekman says that the biggest indicators of lying come from the face, not the body—liars don’t actually fidget more than truthful people, despite what you may have heard.'),
 Document(id='27fbb32b-9b70-4db8-b964-7f3695f11e9e', metadata={}, page_content='Some of

In [62]:
results = collection.similarity_search(query="How to update profile",k=1)

In [63]:
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* How to change relationship status on Facebook?
* Changing your relationship status on Facebook is a straightforward process. First, navigate to your profile by clicking on your name at the top of the Facebook homepage. Then, find and click on the "About" section, which is located near your profile picture. In the "About" section, scroll down to the "Family and Relationships" tab. Next to your current relationship status, you’ll see an “Edit” button. Click on it, select your new relationship status (such as single, in a relationship, married, etc.), and choose whether you want to make this change visible to your friends or the public. Once you've made your selection, click “Save Changes,” and your status will be updated.
How to build trust in a relationship? [{}]


In [66]:
llm = ChatGroq(model="mixtral-8x7b-32768")

In [68]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

In [70]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [72]:
retriever = collection.as_retriever()

In [74]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [76]:
# qa_system_prompt = (
#     "You are an assistant for question-answering tasks. Use "
#     "the following pieces of retrieved context to answer the "
#     "question. If you don't know the answer, just say that you "
#     "don't know. Use three sentences maximum and keep the answer "
#     "concise."
#     "\n\n"
#     "{context}"
# )

In [211]:
qa_system_prompt = (
    "You are a conversational assistant. Use the retrieved context and "
    "the conversation history to answer questions and personalize responses. "
    "If the user shares their name, remember it and use it appropriately."
    "\n\n"
    "{context}"
)


In [213]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [215]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [217]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [219]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = "rag-agent"

In [221]:
react_docstore_prompt = hub.pull("hwchase17/react")

In [224]:
def search_wikipedia(query):
    from wikipedia import summary
    try:
        return summary(query, sentences=2) 
    except:
        return "I couldn't find any information on that."

In [225]:
tools = [
    Tool(
        name="Answer Question through RAG",
        func=lambda input, **kwargs: rag_chain.invoke(
            {
                "input": input,
                "chat_history": kwargs.get("chat_history", []),
            }
        ),
        description="useful for when you need to answer questions about the context",
    ),
    Tool(
        name="wikipedia",
        func=search_wikipedia,
        description="useful when you cannot find answers about the context",
    ),
]

In [226]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_docstore_prompt,
)

In [228]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    handle_parsing_errors=True, 
    verbose=True,
)

In [236]:
from langchain.schema import HumanMessage, AIMessage

In [2]:
chat_history = []
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    response = agent_executor.invoke(
        {"input": query, "chat_history": chat_history})
    print(f"LoveGuru: {response['output']}")
    # print(response)
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=response["output"]))

You:  heelo


NameError: name 'agent_executor' is not defined